# Generate MART and NYES Vector Integration Libraries
- randomly insert vector into location in the genome
- simulate PE reads using BSBolt
- align and try to detect vector integration 

## Notebook Setup

In [1]:
import gzip
import io
import os 
import random
import subprocess
import urllib.request
from tqdm import tqdm_notebook as tqdm

In [2]:
# simulate methylation sequencing data
from BSBolt.Align.AlignmentHelpers import convert_alpha_numeric_cigar, get_mapping_length
from BSBolt.Index.WholeGenomeBuild import WholeGenomeIndexBuild
from BSBolt.Simulate.SimulateMethylatedReads import SimulateMethylatedReads
from BSBolt.Utils.UtilityFunctions import get_external_paths

In [3]:
bt2_path, art_path = get_external_paths()

In [4]:
pwd = os.getcwd() + '/'

In [5]:
ucsc_hg38 = 'https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz'

In [6]:
if not os.path.exists(f'{pwd}SimulationData/hg38.fa.gz'):
    urllib.request.urlretrieve(ucsc_hg38, f'{pwd}SimulationData/hg38.fa.gz')

## Import Sequence Data

In [7]:
# get hg38 reference with lambda phage control attached

hg38 = {}

with gzip.open(f'{pwd}SimulationData/hg38.fa.gz') as genome:
    contig_seq = ''
    chrom = None
    for line in io.BufferedReader(genome):
        processed_line = line.decode('utf-8').strip()
        if '>' == processed_line[0]:
            if chrom:
                hg38[chrom] = contig_seq
            contig_seq = ''
            chrom = processed_line[1:]
        else:
            contig_seq = contig_seq + processed_line
    hg38[chrom] = contig_seq

In [8]:
# import NYES and Mart vector sequences 
mart1 = 'pMSGV1-MART1TCR.txt'
nyes =  'pMSGV1-1G4_A_LY_RetroNYESO1.txt'

In [9]:
vector_seq = {}

for vector in [mart1, nyes]:
    vector_label = vector.replace('-', '_').replace('.txt', '')
    seq = ''
    with open(f'{pwd}SimulationData/{vector}', 'r') as vec_seq:
        for line in vec_seq:
            processed_line = line.strip().replace(' ', '')
            seq = seq + ''.join([base for base in processed_line if not base.isdigit()])
    vector_seq[vector_label] = seq
    

In [10]:
if not os.path.exists(f'{pwd}SimulationData/hg38_sim.fa'):
    out = open(f'{pwd}SimulationData/hg38_sim.fa', 'w')
    for chrom, seq in tqdm(hg38.items()):
        out.write(f'>{chrom}\n')
        out.write(f'{seq}\n')
    for chrom, seq in vector_seq.items():
        out.write(f'>{chrom}\n')
        out.write(f'{seq}\n')
    out.close()

## Simulated vector integration libraries
- select the number of integration events
- select random integration sites
- generate 2 mart and 2 nyes datasets

In [11]:
integration_events = [events + 15 for events in range(10)]

In [12]:
simulation_integration_parameters = {}

for count, vector in enumerate(['pMSGV1_MART1TCR', 'pMSGV1_MART1TCR', 'pMSGV1_1G4_A_LY_RetroNYESO1', 'pMSGV1_1G4_A_LY_RetroNYESO1']):
    sim_label = f'{count}_{vector}'
    int_events = random.sample(integration_events, 1)[0]
    integration_locations = []
    integration_sequences = []
    while len(integration_locations) < int_events:
        int_chrom = random.sample(list(hg38.keys()), 1)[0]
        if len(int_chrom) > 5 or not int_chrom[-1].isdigit():
            continue
        # get chromosome sequence
        chrom_seq = hg38[int_chrom]
        # select simulation position 
        int_pos = random.randint(0, len(chrom_seq))
        # retrieve integration position 
        left_seq = chrom_seq[int_pos - 1000: int_pos]
        right_seq = chrom_seq[int_pos: int_pos + 1000]
        integration_seq = left_seq + vector_seq[vector] + right_seq
        if 'N' in integration_seq or 'n' in integration_seq:
            continue
        integration_sequences.append(integration_seq)
        # save location
        integration_locations.append((int_chrom, int_pos))
    simulation_integration_parameters[sim_label] = integration_locations
    with open(f'{pwd}SimulationData/{sim_label}.fa', 'w') as sim:
        for location, seq in zip(integration_locations, integration_sequences):
            sim.write(f'>{location[0]}_{location[1]}\n')
            sim.write(f'{seq}\n')


In [13]:
def make_directory(directory_path):
    try:
        os.makedirs(directory_path, exist_ok=False)
    except FileExistsError:
        return None

In [14]:
simulation_output = f'{pwd}SimulationData/IntegrationSim/'
make_directory(simulation_output)

In [15]:
for sim_label in simulation_integration_parameters:
    meth_sim = SimulateMethylatedReads(reference_file=f'{pwd}SimulationData/{sim_label}.fa',
                                       art_path=art_path, 
                                       output_path=f'{simulation_output}{sim_label}',
                                       paired_end=True,
                                       read_length=150,
                                       read_depth=40,
                                       insertion_rate1=0.000, insertion_rate2=0.000,
                                       deletion_rate1=0.000, deletion_rate2=0.000,
                                       undirectional=False)
    meth_sim.run_simulation()

Setting Cytosine Methylation
Simulating Illumina Reads
Simulating Methylated Illumina Reads
Finished Simulation
Setting Cytosine Methylation
Simulating Illumina Reads
Simulating Methylated Illumina Reads
Finished Simulation
Setting Cytosine Methylation
Simulating Illumina Reads
Simulating Methylated Illumina Reads
Finished Simulation
Setting Cytosine Methylation
Simulating Illumina Reads
Simulating Methylated Illumina Reads
Finished Simulation


In [16]:
simulation_index = f'{pwd}SimulationData/SimulationIndex/'
if not os.path.exists(simulation_index):
    sim_index = WholeGenomeIndexBuild(reference_file=f'{pwd}SimulationData/hg38_sim.fa', genome_database=simulation_index, bowtie2_path=bt2_path, bowtie2_threads=10)
    sim_index.generate_bsb_database()

## Get control mapping sites for all reads
- get simulated mapping locations
- indentfiy simulated reads the span integration sites

In [17]:
def sam_iterator(sam_file):
    with open(sam_file, 'r') as sam:
        while True:
            line1 = sam.readline()
            if not line1.strip():
                break
            if line1[0] == '@':
                continue
            line2 = sam.readline()
            yield line1.strip().split('\t'), line2.strip().split('\t')

In [18]:
def assess_overlap(span, pos):
    if span[0] < pos < span[1]:
        return True
    return False

In [19]:
integration_reads = {sim_label: {} for sim_label in simulation_integration_parameters}

for sim_label in tqdm(simulation_integration_parameters):
    # need vector length to get the left and right vector boundaries 
    vector_length = len(vector_seq['_'.join(sim_label.split('_')[1:])])
    for line in sam_iterator(f'{simulation_output}{sim_label}.sam'):
        read_len_1 = get_mapping_length(convert_alpha_numeric_cigar(line[0][5]))
        read_len_2 = get_mapping_length(convert_alpha_numeric_cigar(line[1][5]))
        assert line[0][0] == line[1][0]
        assert line[0][2] == line[1][2]
        # normalize coordinates and convert types
        chromosome, vector_pos = line[0][2].split('_')
        vector_pos = int(vector_pos)
        read_1_pos = int(line[0][3])
        read_2_pos = int(line[1][3])
        read_span = [read_1_pos, read_2_pos + read_len_2]
        if line[0][1] == '83':
            read_span = [read_2_pos, read_1_pos + read_len_1]
        assert read_span[0] < read_span[1]
        vector_spans = (assess_overlap(read_span, 1000), assess_overlap(read_span, 1000 + vector_length))
        assert sum(vector_spans) <= 1
        if any(vector_spans):
            vector_boundary = 1000 if vector_spans[0] else 1000 + vector_length
            integration_label = None
            vector_label = None
            if assess_overlap((read_1_pos, read_1_pos + read_len_1), vector_boundary):
                vector_label = ('split_1', chromosome, vector_pos, read_1_pos - vector_boundary)
            elif assess_overlap((read_2_pos, read_2_pos + read_len_2), vector_boundary):
                vector_label = ('split_2', chromosome, vector_pos, read_2_pos - vector_boundary)
            else:
                if assess_overlap((1000, 1000 + vector_length), read_1_pos):
                    vector_label = ('discordant_1', chromosome, vector_pos, read_1_pos - vector_boundary, read_2_pos - vector_boundary)
                else:
                    vector_label = ('discordant_2', chromosome, vector_pos, read_1_pos - vector_boundary, read_2_pos - vector_boundary) 
            if vector_label:
                integration_reads[sim_label][line[0][0]] = vector_label

In [20]:
[len(x) for x in integration_reads.values()]

[1756, 1883, 1972, 2000]

## Align Simulated Bisulfite Sequencing Data

In [21]:
# Align simulated data

sim_alignment_stats = {}

for sim_label in tqdm(simulation_integration_parameters):
    bsbolt_alignment_command = ['python3', '-m', 'BSBolt', 'Align', '-BT2-p', '10', '-F1', f'{simulation_output}{sim_label}_meth_1.fastq', '-F2', f'{simulation_output}{sim_label}_meth_2.fastq',
                                '-O', f'{simulation_output}{sim_label}', '-BT2-local', '-DB', simulation_index, '-discord', '-BT2-score-min', 'G,40,8', '-S']
    sim_align = subprocess.Popen(bsbolt_alignment_command, stdout=subprocess.PIPE, universal_newlines=True)
    alignment_stats = []
    for line in iter(sim_align.stdout.readline, ''):
        alignment_stats.append(line)
    sim_alignment_stats[sim_label] = alignment_stats

In [22]:
sim_alignment_stats

{'0_pMSGV1_MART1TCR': ['Aligning /Users/colinfarrell/Documents/RebisMethylation/VectorInsertionValidation/SimulationData/IntegrationSim/0_pMSGV1_MART1TCR_meth_1.fastq /Users/colinfarrell/Documents/RebisMethylation/VectorInsertionValidation/SimulationData/IntegrationSim/0_pMSGV1_MART1TCR_meth_2.fastq\n',
  'Alignment Complete: Time 0:00:28\n',
  '------------------------------\n',
  'Total Reads: 19840\n',
  'Reads Mapped 0 Times: 22, 9 Multi-reference\n',
  'Reads Mapped 1 Time: 8018\n',
  'Reads Mapped >1 Times: 10642\n',
  '------------------------------\n',
  'Reads Mapped Discordantly 1 Time: 108\n',
  'Reads Mapped Discordantly >1 Times: 976\n',
  'Reads with Mixed Mapping 1 Time: 29\n',
  'Reads with Mixed Mapping >1 Times: 45\n',
  '------------------------------\n',
  'Mappability: 99.889 %\n',
  '------------------------------\n',
  'Reads Mapped to Watson_C2T: 9927\n',
  'Reads Mapped to Crick_C2T: 9890\n',
  'Reads Mapped to Watson_G2A: 1\n',
  'Reads Mapped to Crick_G2A: 0\

### Get Dicordant and Split Vector Reads

In [248]:
def stream_mapped_reads(file_path, included_flag=None, excluded_flag=None):
    """ process reads streamed using samtools view, samtools must be on
        path for this to work """
    stream_command = ['samtools', 'view']
    if included_flag:
        stream_command.extend(['-F', str(included_flag)])
    if excluded_flag:
        stream_command.extend(['-f', str(excluded_flag)])
    stream_command.append(file_path)
    read_stream = subprocess.Popen(stream_command, stdout=subprocess.PIPE, universal_newlines=True)
    for line in iter(read_stream.stdout.readline, ''):
        QNAME, FLAG, RNAME, POS, MAPQ, CIGAR, RNEXT, PNEXT, TLEN, SEQ, QUAL, *SAM_TAGS = line.strip().split('\t')
        alignment_score = None
        mapping_reference = None
        for tag in SAM_TAGS:
            if tag[0:3] == 'AS:':
                alignment_score = tag.split(':')[-1]
            elif tag[0:4] == 'XO:Z':
                mapping_reference = tag.split(':')[-1]
        yield QNAME, FLAG, RNAME, int(POS), CIGAR, int(alignment_score), mapping_reference


def get_spanning_reads(file_path: str = None, plasmid_names: set = None) -> dict:
    mapped_reads = {}
    for sam_read in stream_mapped_reads(file_path, included_flag=4):
        QNAME, FLAG, RNAME, POS, CIGAR, alignment_score, mapping_reference = sam_read
        plasmid_read = RNAME in plasmid_names
        if QNAME not in mapped_reads:
            mapped_reads[QNAME] = [[sam_read], plasmid_read]
        else:
            if plasmid_read:
                mapped_reads[QNAME][0].append(sam_read)
                mapped_reads[QNAME][1] = plasmid_read
            else:
                mapped_reads[QNAME][0].append(sam_read)
    plasmid_reads = {}
    for qname, read_group in mapped_reads.items():
        if read_group[1]:
            for read in read_group[0]:
                if read[0][0:3] == 'chr':
                    plasmid_reads[qname] = read_group[0]
                    break
    return plasmid_reads


In [249]:
sample_spanning_reads = {}
sample_read_stats = {}

for sim_label in tqdm(simulation_integration_parameters):
    plasmid_reads = get_spanning_reads(f'{simulation_output}{sim_label}.sorted.bam', {'pMSGV1_MART1TCR', 'pMSGV1_1G4_A_LY_RetroNYESO1'})
    int_read_count = 0
    non_int_read_count = 0
    for read in plasmid_reads:
        if read in integration_reads[sim_label]:
            int_read_count += 1
        else:
            non_int_read_count += 1
    sample_spanning_reads[sim_label] = plasmid_reads
    observed_read_stats = dict(integration_reads=int_read_count, non_integration_reads=non_int_read_count, 
                               total_simulated_integration_reads=len(integration_reads[sim_label]), total_spanning_reads=len(plasmid_reads))
    sample_read_stats[sim_label] = observed_read_stats

In [250]:
sample_read_stats

{'0_pMSGV1_MART1TCR': {'integration_reads': 1250,
  'non_integration_reads': 15504,
  'total_simulated_integration_reads': 1756,
  'total_spanning_reads': 16754},
 '1_pMSGV1_MART1TCR': {'integration_reads': 1352,
  'non_integration_reads': 16402,
  'total_simulated_integration_reads': 1883,
  'total_spanning_reads': 17754},
 '2_pMSGV1_1G4_A_LY_RetroNYESO1': {'integration_reads': 1417,
  'non_integration_reads': 17303,
  'total_simulated_integration_reads': 1972,
  'total_spanning_reads': 18720},
 '3_pMSGV1_1G4_A_LY_RetroNYESO1': {'integration_reads': 1431,
  'non_integration_reads': 17312,
  'total_simulated_integration_reads': 2000,
  'total_spanning_reads': 18743}}

In [251]:
# clean reads mapping to different vector

for sim_label in tqdm(simulation_integration_parameters):
    vector = '_'.join(sim_label.split('_')[1:])
    plasmid_reads = sample_spanning_reads[sim_label]
    cleaned_plasmid_reads = {}
    for read_name, read_group in plasmid_reads.items():
        cleaned_group = []
        vector_mapping = False
        for read in read_group:
            if read[2][0:3] == 'chr':
                cleaned_group.append(read)
            elif read[2] == vector:
                vector_mapping = True
                cleaned_group.append(read)
        if vector_mapping:
            cleaned_plasmid_reads[read_name] = cleaned_group
    sample_spanning_reads[sim_label] = cleaned_plasmid_reads

In [252]:
from typing import Dict, List, Tuple

In [320]:
from collections import namedtuple

class ProcessVectorSpanningReads:
    """Indentify high quality reads or read pairs that span a vector of interest and the genome."""

    def __init__(self, read_uniqueness_threshold: float = 1.1, read_mapping_threshold: float = 0.9,
                 multibase_threshold: float = 0.1):
        self.first_read = {'65', '67', '73', '81', '89', '97', '113', '115', '321', '323', '345', '329', '369', '371'}
        self.fr_reference = {'W_C2T', 'C_G2A'}
        self.proper_pair = {'131', '435', '371', '179', '67', '115', '323', '387'}
        self.formatted_read = namedtuple('f_read', ['qname', 'rname', 'flag', 'mapping_ref', 'left_ref', 'right_ref', 
                                                    'left_query', 'right_query', 'alignment_score', 'matched_base_pos'])
        self.read_uniqueness_threshold = read_uniqueness_threshold
        self.read_mapping_threshold = read_mapping_threshold
        self.multibase_threshold = multibase_threshold

    def get_integration_sites(self, read_group: list = None, vector: str = None):
        first_reads, second_reads = self.get_paired_reads(read_group)
        group_1 = self.process_read_mapping(first_reads)
        group_2 = self.process_read_mapping(second_reads)
        g1_vector, g1_genome, g1_split = self.assess_alignment_contigs(group_1, vector)
        g2_vector, g2_genome, g2_split  = self.assess_alignment_contigs(group_2, vector)
        # discard read groups with conflicting integration site information
        if g1_split and g2_split:
            return None
        # discard read group with only chromosome or vector mapping information
        if not any((g1_vector, g2_vector)) or not any((g1_genome, g2_genome)):
            return None
        # discordant reads 
        if not g1_split and not g2_split:
            # read information -> QNAME, CHROM, mapping_ref, l_reference_pos, r_reference_pos, l_query_pos, r_query_pos, Alignment_Score, list[matched_base_position: int]
            return self.process_discordant_int(group_1, group_2, g1_genome)
        return self.process_split_int(group_1, group_2, g1_split, g1_genome, g2_genome, vector)
        
    def process_split_int(group_1: list, group_2: list, g1_split: bool, g1_genome: bool, g2_genome: bool, vector: str):
        split_group = group_1 if g1_split else group_2
        supporting_group = group1 if not g1_split else group_2
        supporting_genome = g1_genome if not g1_split else g2_genome
        genome_split, vector_split = None, None
        for read in split_group:
            if read.rname == vector:
                vector_split = read
            else:
                genome_split = read
        ref_pos = genome_split.left_ref if genome_split.left_query > vector_split.left_query else genome_split.right_ref
        if not supporting_group:
            return 'split_single', genome_split.qname, genome_split.rname, ref_pos, genome_split.alignmet_score, vector_split.alignment_score
        else:
            if supporting_genome:
                if supporting_group[0][1] != genome_split[0][1]:
                    return None
                return 'split_paired', genome_split.qname, genome_split.rname, ref_pos, genome_split.alignmet_score + supporting_genome.alignment_score, vector_split.alignment_score
            else:
                return 'split_paired', genome_split.qname, genome_split.rname, ref_pos, genome_split.alignmet_score, vector_split.alignment_score + supporting_genome.alignment_score

        
    def process_discordant_int(self, group_1, group_2, g1_genome):
        read_1 = group_1[0]
        read_2 = group_2[0]
        if read_1.flag in self.proper_pair:
            return None
        if g1_genome:
            ref_pos = read_1.right_ref if read_1.flag in self.fr_reference else read_1.left_ref
            return 'discord_1', read_1.qname, read_1.rname, ref_pos, read_1.alignment_score, read_2.alignment_score
        else:
            ref_pos = read_2.left_ref if read_2.flag in self.fr_reference else read_2.right_ref
            return 'discord_2', read_2.qname, read_2.rname, ref_pos, read_2.alignment_score, read_1.alignment_score
        

    def process_read_groups(self, group_1, group_2):
        if group_1[6] and group_2[6]:
            return None
        pass

    def assess_alignment_contigs(self, read_group: list, vector: str) -> (bool, bool):
        vector_mapping, genome_mapping = False, False
        genome_contigs = []
        for read in read_group:
            if read.rname == vector:
                vector_mapping = True
            else:
                genome_contigs.append(read.rname)
                genome_mapping = True
        if len(genome_contigs) > 1:
            genome_mapping = False
            vector_mapping = False
        return vector_mapping, genome_mapping, all((vector_mapping, genome_mapping))

    def get_paired_reads(self, read_group):
        first_reads, second_reads = [], []
        for read in read_group:
            if read[1] in self.first_read:
                first_reads.append(read)
            else:
                second_reads.append(read)
        return first_reads, second_reads

    def process_read_mapping(self, read_paired):
        processed_reads = []
        for read in read_paired:
            qname, rname, flag, pos, cigar, alignment_score, mapping_ref = read
            cigar_tuple = convert_alpha_numeric_cigar(cigar)
            # read information -> l_query_pos, r_query_pos, l_reference_pos, r_reference_pos, list[matched_base_position: int]
            l_pos, r_pos, reference_pos, matched_base_pos = self.get_mapped_bases(cigar_tuple, pos)
            # read information -> QNAME, flag, CHROM, mapping_ref, l_ref_pos, r_ref_pos, l_query_pos, r_query_pos, Alignment_Score, list[matched_base_position: int]
            formatted_read = self.formatted_read(qname, flag, rname, mapping_ref, pos, reference_pos, l_pos, r_pos, alignment_score, matched_base_pos)
            processed_reads.append(formatted_read)
        processed_reads.sort(key = lambda x: x.alignment_score, reverse=True)
        if not processed_reads:
            return processed_reads
        if len(processed_reads) < 2:
            return processed_reads
        else:
            primary_reads = [processed_reads[0]]
            for read in processed_reads[1:]:
                for p_read in primary_reads:
                    if self.get_duplication_proportion(read.matched_base_pos, p_read.matched_base_pos) > self.multibase_threshold:
                        duplicate_read = True
                        break
                if not duplicate_read:
                    primary_reads.append(read)
        return primary_reads
    
    @staticmethod
    def get_duplication_proportion(read_bases, comparison_bases):
        return len([base for base in read_bases if base in comparison_bases]) / len(read_bases)

    def get_mapped_bases(self, cigar_tuple: tuple, reference_position: int) -> (int, int, list):
        matched_base_positions = []
        reference_consumers = {0, 2, 3, 7, 8}
        query_consumers = {0, 1, 4, 7, 8}
        # set relative to genomic position so add reference start and one since capturing the first base
        query_position = 0
        left_mapped_pos, right_mapped_pos = None, None
        for cigar_type, cigar_count in cigar_tuple:
            if cigar_type in reference_consumers and cigar_type in query_consumers:
                for _ in range(cigar_count):
                    if not left_mapped_pos:
                        left_mapped_pos = query_position
                    right_mapped_pos = query_position
                    matched_base_positions.append(query_position)
                    query_position += 1
                    reference_position += 1
            elif cigar_type in query_consumers and cigar_type not in reference_consumers:
                query_position += cigar_count
            elif not cigar_type in query_consumers and cigar_type in reference_consumers:
                reference_position += 1
        return left_mapped_pos, right_mapped_pos, reference_position, matched_base_positions


In [321]:
test_processor = ProcessVectorSpanningReads(read_uniqueness_threshold=1.2, read_mapping_threshold=0.7,
                                            multibase_threshold=0.2)

In [322]:
test_group = [('chr15_93920560-2110', '435', 'chr14', 22052763, '37S4M1I108M', 216, 'C_C2T'), ('chr15_93920560-2110', '371', 'chr14', 22052867, '150M', 300, 'C_C2T'), ('chr15_93920560-2110', '179', 'pMSGV1_1G4_A_LY_RetroNYESO1', 7, '150M', 300, 'C_C2T'), ('chr15_93920560-2110', '115', 'pMSGV1_1G4_A_LY_RetroNYESO1', 149, '150M', 300, 'C_C2T')]

In [323]:
test_processor.get_integration_sites(test_group, vector='pMSGV1_1G4_A_LY_RetroNYESO1')

In [324]:
x = {67: 131, 323: 387, 115: 179, 371: 435,
                      131: 67, 387: 323, 179: 115, 435: 371}
print({str(y) for y in x.keys()})

{'131', '435', '371', '179', '67', '115', '323', '387'}


In [331]:
test_processor = ProcessVectorSpanningReads(read_uniqueness_threshold=1.2, read_mapping_threshold=0.7,
                                            multibase_threshold=0.2)

good_call = 0
bad_call = 0

for sim_label in tqdm(simulation_integration_parameters):
    vector = '_'.join(sim_label.split('_')[1:])
    for read_label, read_group in sample_spanning_reads[sim_label].items():
        control_info = integration_reads[sim_label].get(read_label, False)
        called_int = test_processor.get_integration_sites(read_group, vector=vector)
        if control_info and called_int:
            good_call += 1
        elif not control_info and called_int:
            print(read_group)
            print(sim_label, called_int)
            bad_call += 1

[('chr19_14095219-1158', '65', 'chr14', 22547510, '150M', 300, 'W_C2T'), ('chr19_14095219-1158', '321', 'pMSGV1_1G4_A_LY_RetroNYESO1', 407, '150M', 300, 'W_C2T'), ('chr19_14095219-1158', '129', 'pMSGV1_1G4_A_LY_RetroNYESO1', 764, '150M', 300, 'W_C2T')]
2_pMSGV1_1G4_A_LY_RetroNYESO1 ('discord_1', 'chr19_14095219-1158', 'chr14', 22547510, 300, 300)
[('chr16_85134609-476', '433', 'chr14', 22547575, '150M', 300, 'C_C2T'), ('chr16_85134609-476', '177', 'pMSGV1_1G4_A_LY_RetroNYESO1', 472, '150M', 300, 'C_C2T'), ('chr16_85134609-476', '113', 'pMSGV1_1G4_A_LY_RetroNYESO1', 830, '150M', 295, 'C_C2T')]
2_pMSGV1_1G4_A_LY_RetroNYESO1 ('discord_2', 'chr16_85134609-476', 'chr14', 22547725, 300, 295)
[('chr15_93920560-350', '433', 'chr14', 22547704, '76M74S', 152, 'C_C2T'), ('chr15_93920560-350', '369', 'chr7', 142451152, '150M', 300, 'C_C2T'), ('chr15_93920560-350', '177', 'pMSGV1_1G4_A_LY_RetroNYESO1', 601, '150M', 300, 'C_C2T'), ('chr15_93920560-350', '113', 'pMSGV1_1G4_A_LY_RetroNYESO1', 953, '15

In [327]:
good_call

2620

In [328]:
sum([len(x) for x in integration_reads.values()])

7611

In [329]:
bad_call

60